<a href="https://colab.research.google.com/github/Rizwankaka/Agentic-AI-/blob/main/Langchain/sql_db_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building an Intelligent SQL Query Assistant with LangChain and Groq

## Introduction
This notebook shows how to create an intelligent SQL query assistant using LangChain, Groq LLM, and SQLite. The system is designed to convert natural language questions into SQL queries and provide detailed explanations of the results, making database interactions more accessible to non-technical users.

## Goal
The primary objectives of this notebook are:
1. Set up a database environment by converting CSV data (salary information) into a SQLite database
2. Create an intelligent SQL agent using LangChain that can:
   - Interpret natural language questions about employee salary data
   - Generate appropriate SQL queries automatically
   - Execute queries against the database
   - Provide human-readable explanations of the results
3. Demonstrate the agent's capabilities by analyzing employee compensation data, including metrics like base salary and longevity pay across different grade levels

## Technical Components
- **LangChain**: Framework for building LLM applications
- **Groq**: Large Language Model provider (using llama-3.3-70b-versatile model)
- **SQLite**: Lightweight database for storing and querying salary data
- **Pandas**: Data manipulation and CSV handling
- **SQLAlchemy**: SQL toolkit and ORM

The notebook showcases a practical implementation of natural language to SQL conversion, making database querying more accessible while maintaining the ability to perform complex analytical tasks on employee compensation data.

In [4]:
%pip install -qU pandas pyodbc tabulate langchain langchain-community langchain-core langchain-experimental SQLAlchemy langchain-groq

In [5]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [6]:
from langchain_groq import ChatGroq
from sqlalchemy import create_engine

In [10]:
llm_name = "llama-3.3-70b-versatile"
model = ChatGroq(model=llm_name)

import pandas as pd
from langchain.agents import create_sql_agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase

# create a db from csv file

# Path to your SQLite database file
database_file_path = "./db/salary.db"


# Create an engine to connect to the SQLite database
# SQLite only requires the path to the database file
engine = create_engine(f"sqlite:///{database_file_path}")
file_url = "/content/salaries_2023.csv"
os.makedirs(os.path.dirname(database_file_path), exist_ok=True)
df = pd.read_csv(file_url).fillna(value=0)
df.to_sql("salaries_2023", con=engine, if_exists="replace", index=False)

# print(f"Database created successfully! {df}")

# Part 2: Prepare the sql prompt
MSSQL_AGENT_PREFIX = """

You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct {dialect} query
to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most {top_k} results.
- You can order the results by a relevant column to return the most
interesting examples in the database.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it.If you get an error
while executing a query,rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.)
to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- Your response should be in Markdown. However, **when running  a SQL Query
in "Action Input", do not include the markdown backticks**.
Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- If the question does not seem related to the database, just return
"I don\'t know" as the answer.
- Only use the below tools. Only use the information returned by the
below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the
tools below.
- as part of your final answer, please include the SQL query you used in json format or code format

## Tools:

"""

MSSQL_AGENT_FORMAT_INSTRUCTIONS = """

## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input:
SELECT TOP (10) [base_salary], [grade]
FROM salaries_2023

WHERE state = 'Division'

Observation:
[(27437.0,), (27088.0,), (26762.0,), (26521.0,), (26472.0,), (26421.0,), (26408.0,)]
Thought:I now know the final answer
Final Answer: There were 27437 workers making 100,000.

Explanation:
I queried the `xyz` table for the `salary` column where the department
is 'IGM' and the date starts with '2020'. The query returned a list of tuples
with the bazse salary for each day in 2020. To answer the question,
I took the sum of all the salaries in the list, which is 27437.
I used the following query

```sql
SELECT [salary] FROM xyztable WHERE department = 'IGM' AND date LIKE '2020%'"
```
===> End of Example

"""


db = SQLDatabase.from_uri(f"sqlite:///{database_file_path}")
toolkit = SQLDatabaseToolkit(db=db, llm=model)

QUESTION = """What is the average longevity pay for employees in grade M3? Which grade has the highest average base salary?"
"""
sql_agent = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions=MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=model,
    toolkit=toolkit,
    top_k=30,
    verbose=True,
)

res = sql_agent.invoke(QUESTION)





> Entering new SQL Agent Executor chain...
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: salaries_2023I have the list of tables, and I can see that there is a table called "salaries_2023". This table is likely to contain information about employee salaries, including longevity pay and base salary. I should query the schema of this table to see what columns it contains.

Action: sql_db_schema
Action Input: salaries_2023
CREATE TABLE salaries_2023 (
	"Department" TEXT, 
	"Department_Name" TEXT, 
	"Division" TEXT, 
	"Gender" TEXT, 
	"Base_Salary" FLOAT, 
	"Overtime_Pay" FLOAT, 
	"Longevity_Pay" FLOAT, 
	"Grade" TEXT
)

/*
3 rows from salaries_2023 table:
Department	Department_Name	Division	Gender	Base_Salary	Overtime_Pay	Longevity_Pay	Grade
ABS	Alcohol Beverage Services	ABS 85 Administration	M	175873.0	0.0	0.0	M2
ABS	Alcohol Beverage Services	ABS 85 Admin